<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/Query_CTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsquedas WOS+SCI+SCP+PTJ+CTR para UdeA

Búsquedas en bases bibligráficas  
* Web of Science (WOS), 
* Scielo (SCI)
* Scopus  (SCP)
* Puntaje (UDEA)
* Center (CTR)
de los artículos científicos de la UdeA

La base de datos se creó con:

[WOS_SCI_SCP_PTJ_CTR.ipynb](./WOS_SCI_SCP_PTJ_CTR.ipynb)

In [25]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus fuzzywuzzy[speedup] > /dev/null

## functions

In [26]:
import pandas as pd
import wosplus as wp
pd.set_option('display.max_colwidth',200)
from venn import draw_venn, generate_colors
import numpy as np
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz
import re
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import unidecode
import itertools
import sys
idc='CÉDULA'
# GENERAL PORPOSE FUNCTIONS
def split_names(s):
    """
    Extract the parts of the full name s.
    
    Works with:
    ----
        s='LA ROTTA FORERO DANIEL ANDRES'
        s='MONTES RAMIREZ MARIA DEL CONSUELO'
        s='CALLEJAS POSADA RICARDO DE LA MERCED'
        s='DE LA CUESTA BENJUMEA MARIA DEL CARMEN'
        s='JARAMILLO OCAMPO NICOLAS CARLOS MARTI'
        s='RESTREPO QUINTERO DIEGO ALEJANDRO'
        s='RESTREPO QUINTERO DIEGO'
        s='RESTREPO DIEGO'
    Fails with: 
    ----
        s='RANGEL MARTINEZ VILLAL ANDRES MAURICIO'
        s='RESTREPO DIEGO ALEJANDRO'
    """
    s=s.title()
    sl=re.sub('(\s\w{1,3})\s',r'\1-',s,re.UNICODE)
    sl=re.sub('^(\w{1,3})\s',r'\1-' ,sl,re.UNICODE)
    #if sl.find('-')>-1:
    sll=[s.replace('-',' ') for s in sl.split()]
    if len(s.split())==2:
        sll=[s.split()[0]]+['']+[s.split()[1]]
    #
    d={'NOMBRE COMPLETO' : ' '.join(sll[2:]+sll[:2]),
     'PRIMER APELLIDO' : sll[0], 
     'SEGUNDO APELLIDO': sll[1], 
     'NOMBRES'         :' '.join(sll[2:]), 
     'INICIALES'       :' '.join( [i[0]+'.' for i in ' '.join(sll[2:]).split() ] )
    }
    return d

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel/json or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [27]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs
produccion_fecha_vig_2003_2018.xlsx=1WbtX4K__TTLxXRjuLvqUYz9tuHCIlS5v
producción_reconocida_2002_2016_doi.xlsx=0BxoOXsn2EUNIY2lzVmNMT0VXaGs
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x

Overwriting drive.cfg


##  Load data bases

In [28]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')

In [29]:
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'
tmp=drive_files.load_biblio(UDEAjsonfile,compression='gzip')
UDEA=drive_files.biblio['WOS'].copy().reset_index(drop=True)

In [6]:
AU=drive_files.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

In [7]:
PTJ=drive_files.read_drive_excel('produccion_fecha_vig_2003_2018.xlsx')
AU_PTJ=PTJ[['cedula','nombre']].drop_duplicates()

In [ ]:
PTJ[:2][['nombre','título']]

In [8]:
LOCAL=True
if LOCAL:
    UDEA.to_json('WOS_SCI_SCP_PTJ_CTR.json.gz',orient='records')
    AU.to_json('AU.json')
    AU_PTJ.to_json('AU_PTJ.json',orient='records')

In [9]:
LOCAL_LOAD=False
if LOCAL_LOAD:
    UDEA=pd.read_json('WOS_SCI_SCP_PTJ_CTR.json.gz')
    AU=pd.read_json('AU.json').reset_index(drop=True)
    AU_PTJ=pd.read_json('AU_PTJ.json').reset_index(drop=True)

In [10]:
#UDEA=pd.read_json('/home/restrepo/Downloads/WOS_SCI_SCP_PTJ_CTR.json.gz').reset_index(drop=True)

In [11]:
UDEA[UDEA['UDEA_authors']!=''].reset_index(drop=True).shape

(15642, 184)

In [12]:
UDEA.shape

(15642, 184)

## Normalizations

### Relacionada con autores

Normalize `'AU'`

In [13]:
UDEA['AU']=UDEA['AU'].apply(lambda l: l if re.search('\n$',l) 
                 else 
                 '\n'.join(
            [
    re.sub( '([\w\-\s]+)(\s[\w]\.)',r'\1,\2',s ,re.UNICODE   ) 
                 for s in l.split(', ')]+['']
           ).replace('.','') )

In [14]:
UDEA[~UDEA['AU'].str.contains('\n$')].shape

(0, 184)

Normalize `'authors_WOS'`

In [15]:
#Specific entries
#('mcewen, juan g', 65) → mc ewen
#'hincape, me' → hincapie
def fix_WOS_author_Trujilloii(l):
    for i in range(len(l)):
        if l[i].get('WOS_author') and l[i].get('WOS_author')=='Trujilloii, S. B.':
            l[i]={'WOS_author': 'Trujillo, S. B.',
                  'affiliation': [
                    'Facultad de Medicina, Sede de Investigación Universitaria (SIU), Univ Antioquia, Medellín, Colombia'
                   ],
                  'i': 3}
        if l[i].get('WOS_author') and l[i].get('WOS_author')=='Manuel, J':
            l[i]={'WOS_author':'Senior Sanchez, Juan Manuel'}
            
        if l[i].get('WOS_author') and l[i].get('WOS_author')=='Balthazar, Vital':
            l[i]={'WOS_author': 'Baltazar, Vital',
                  'affiliation': ['Univ Antioquia, Colombia.'],
                  'i': 4}
        if l[i].get('WOS_author') and l[i].get('WOS_author')=='Oliviera-Angel, M':
            l[i]={'WOS_author': 'Olivera-Angel, M',
              'affiliation': ['Univ Antioquia, Fac Ciencias Agrarias Fisiol & Biotecnol Reprod, Antioquia, Colombia.']}            
    return l
UDEA['authors_WOS']=UDEA['authors_WOS'].apply( fix_WOS_author_Trujilloii )

In [16]:
def add_affilitions_to_authors_WOS(row_authors_WOS,row_C1,row_SCP_Affiliations):
    l=row_authors_WOS
    #Get rid of multiple author affiliations
    #TODO: Count multiple author affiliations
    row_C1=re.sub(r'\[[\w\.\s,]+\n[\w\.\s,]+\]','',row_C1,re.UNICODE)
    afwos=row_C1.strip().split('\n')
    afscp=row_SCP_Affiliations.strip().split('; ')
    if len(afwos)==1:
        af=re.sub('\[.*\]\s*','',afwos[0])
        for i in range( len(l) ):
            if not l[i].get('affiliation'):
                l[i].update({'affiliation':[af]})

    if len(afwos)==len(l):
        for i in range( len(l) ):
            if not l[i].get('affiliation'):
                af=re.sub('\[.*\]\s*','',afwos[i])
                l[i].update({'affiliation':[af]})
            
    elif len(afscp)==len(l):
        for i in range( len(l) ):
            if not l[i].get('affiliation'):
                l[i].update({'affiliation':[afscp[i]]})
    return l

In [17]:
def missing_authors_WOS(row):
    '''
    if row 'authors_WOS' is empty, try to fill it
    with AU and C1.
    if not empty just return it
    '''
    l=[]
    if len(row['authors_WOS'])==0:
        if row['AU']:
            l=[ {'WOS_author':a} for a in row['AU'].strip().split('\n') ]
        #Affiliations:
        row_C1=row['C1']
        row_SCP_Affiliations=row['SCP_Affiliations']
        l=add_affilitions_to_authors_WOS(l,row_C1,row_SCP_Affiliations)
    else:
        l=row['authors_WOS']
                     
    return l

In [18]:
UDEA['authors_WOS']=UDEA.apply(missing_authors_WOS,axis='columns')

In [19]:
def check_AU_authors(AU1,AU2,fuzz_partial_ratio=60):
    '''
    Check if two authors in the WOS AU format are the same
    '''
    AU1_eq_AU2=False
    #Check last name
    au1=AU1.split(', ')
    au2=AU2.split(', ')
    au1=[unidecode.unidecode(a.lower()) for a in au1]
    au2=[unidecode.unidecode(a.lower()) for a in au2]
    ln=np.intersect1d(au1[0].split(),au2[0].split()).shape[0]
    if ln>0:
        fn=np.intersect1d(  
            [ l[0] for l in au1[-1].split()],
            [l[0] for l in au2[-1].split()] ).shape[0]
        if fn>0:
            if fuzz.partial_ratio( AU1,AU2 )>fuzz_partial_ratio:
                AU1_eq_AU2=True
    return AU1_eq_AU2

def AU_to_authors_WOS(row,DEBUG=False):
    '''
     Compare 'WOS_author' → 'authors_WOS' with
             'AU' splitted list
     and fill for the missing 'authors_WOS' and
                              'affiliations'
    '''
    auwos=[d.get('WOS_author') for d in row['authors_WOS']]
    aurow=row['AU'].strip().split('\n')
    if len(auwos)<len(aurow):
        for au in aurow:
            NEW_AU=True
            for ws in auwos:
                if DEBUG:
                    print(au,'::',ws,'→',  check_AU_authors( au,ws) )
                if check_AU_authors( au,ws):
                    NEW_AU=False
                    break
            if NEW_AU:
                row['authors_WOS'].append(  
                    {'WOS_author':au})
        row['authors_WOS']=add_affilitions_to_authors_WOS(row['authors_WOS'],
                                       row['C1'],
                                       row['SCP_Affiliations'])
    return row['authors_WOS']

In [20]:
UDEA['authors_WOS']=UDEA.apply(AU_to_authors_WOS,axis='columns')

Normalize `'UDEA_authors'`

In [21]:
def fix_long_names(l):
    if len(l)>0:
        for i in range( len(l) ):
            if  l[i].get('full_name') and len(l[i].get('full_name').split())>4:
                dd=split_names(l[i].get('full_name'))
                l[i].update(dd)
    return l

In [22]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply( fix_long_names  )

In [23]:
def drop_duplicates(l):
    """
    Find duplicates in UDEA_authors by 'CÉDULA'
    """
    if len(l)>1:
        #Try to find duplicates j-times
        for j in range( int((len(l)+1)/2)   ):
            c=[]
            #In each try remove duplicated
            for i in range(len(l)):
                if l[i].get('CÉDULA') in c:
                    l.pop(i)
                    break
                else:
                    c.append(l[i].get('CÉDULA'))
            #Exit when not more duplicate found
            if not c:
                break
    return l

In [24]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(drop_duplicates)

In [25]:
def fix_NOMBRE_COMPLETOS(l):
    #for d in l:
    for i in range(len(l)):
        if not l[i].get('NOMBRE COMPLETO'):
            d['NOMBRE COMPLETO']=''
            if l[i].get('NOMBRES'):
                l[i]['NOMBRE COMPLETO']=l[i]['NOMBRE COMPLETO']+l[i].get('NOMBRES')
            if l[i].get('PRIMER APELLIDO'):
                l[i]['NOMBRE COMPLETO']=l[i]['NOMBRE COMPLETO']+' '+l[i].get('PRIMER APELLIDO')
            if l[i].get('SEGUNDO APELLIDO'):
                l[i]['NOMBRE COMPLETO']=l[i]['NOMBRE COMPLETO']+' '+l[i].get('SEGUNDO APELLIDO')            
    return l

In [26]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(fix_NOMBRE_COMPLETOS)

In [27]:
dfid=UDEA[['UDEA_nombre','UDEA_cedula']]
dfid=dfid[dfid['UDEA_cedula'].apply(lambda n: n!=0 if isinstance(n,int) else False)].reset_index(drop=True)
dfid=dfid[dfid['UDEA_nombre']!=''].drop_duplicates().reset_index(drop=True)
dfid.shape

def get_cedula(l,fullnames_with_id=dfid):
    #for d in l:
    for i in range(len(l)):
        if not l[i].get('CÉDULA') and l[i].get('full_name'):
            try:
                l[i]['CÉDULA']=dfid[ ( dfid['UDEA_nombre']==l[i].get('full_name')) 
                            & ( dfid['UDEA_cedula']!=0 ) 
                            ].reset_index(
                          drop=True)['UDEA_cedula'].loc[0]
            except KeyError:
                pass
    return l

In [28]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(get_cedula)

In [29]:
def get_missing_cedula(l,AU=AU):
    '''
    AU: Data Frame with author info
    Added Facultad y afiliacion WOS
    '''
    for i in range(len(l)):
        if not l[i].get('CÉDULA'):
            au=AU[AU['NOMBRE COMPLETO']==l[i].get('NOMBRE COMPLETO')
                 ].reset_index(drop=True)
            if not au.empty:
                au=au.loc[0]
                l[i]['CÉDULA']=au['CÉDULA']
                if not l[i].get('DEPARTAMENTO'):
                    l[i]['DEPARTAMENTO']=au['DEPARTAMENTO']
                if not l[i].get('FACULTAD'):
                    l[i]['FACULTAD']=au['FACULTAD']
                if not l[i].get('NOMBRES'):
                    l[i]['NOMBRES']=au['NOMBRES']
                if not l[i].get('PRIMER APELLIDO'):
                    l[i]['PRIMER APELLIDO']=au['PRIMER APELLIDO']
                if not l[i].get('SEGUNDO APELLIDO'):
                    l[i]['SEGUNDO APELLIDO']=au['SEGUNDO APELLIDO']
                if not l[i].get('INICIALES'):
                    s=au['NOMBRES']
                    l[i]['INICIALES']=' '.join(
                        [ I[0]+'.' for I in s.split() ])
    return l

In [30]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(get_missing_cedula)

In [31]:
def fix_cedula(l,df=AU_PTJ,full_name='nombre',identification='cedula'):
    '''
    Get missing identification from an external database with
    the full name in the proper format:
     `full name`: LAST NAME FIRST NAME
    For each row, rebuild the list of dictionaries
    and if CÉDULA is missing try to fix it
    '''
    for i in range(len(l)):
        if not l[i].get('CÉDULA'):
            try:
                l[i]['CÉDULA']=df[df[nombre]==l[i].get('full_name')
                     ].reset_index(drop=True).loc[
                    0,identification]
            except:
                pass
    return l

In [32]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(fix_cedula)

In [33]:
UDEA['C1']=UDEA['C1'].str.replace('deAntioquia','de Antioquia')

In [34]:
def add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,df,j):
    if not l[i].get('WOS_author'):
        l[i]['WOS_author']=[df.loc[j,'WOS_author']]
    if 'UDEA_affiliation' in df.columns:
        if not l[i].get('UDEA_affiliation'):
            l[i]['UDEA_affiliation']=df.loc[j,'affiliation']
    return l[i]

def compare_names(l,i,ll,j,smin=70,DEBUG=False):
    '''
    Compare dictionaries with UDEA_authors keys
    with DataFrame ll authors_WOS columns
    1. Select first matching first last name
       a. If failed select first mathcing second last name
          with extra quality check in full name
    2. First mathching names
       a. If failed first matching initials
    '''
    #j=-1 initialized in main function
    if DEBUG: print('i:',i)
    #Find First last name
    r=process.extractOne( unidecode.unidecode( l[i].get('PRIMER APELLIDO').lower() ),  
                       ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                    ).str.split(', ').str[0],scorer=fuzz.partial_ratio )
    if DEBUG: print(r)
    if r[1]==100:
        j=r[2]
        
    #If failed, try with the second last name
    if r[1]<100:
        r=process.extractOne( unidecode.unidecode( l[i].get('SEGUNDO APELLIDO').lower() ),  
                           ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                        ).str.split(', ').str[0],scorer=fuzz.partial_ratio )
        #Additional quality check
        j=r[2]
        s=fuzz.token_set_ratio( 
            unidecode.unidecode( l[i].get('NOMBRE COMPLETO').lower().replace('.','') ) ,
            unidecode.unidecode(ll['WOS_author'].loc[j].lower().replace('.','')  ) )
        if DEBUG: print(r)
        #print(s)
        if r[1]==100 and s>smin:
            j=r[2]
        else:
            j=-1
        
    #For names filter Initials:
    kk=ll[ll['WOS_author'].str.lower().apply(unidecode.unidecode
                        ).str.split(
                    ', ').str[-1].str.replace(
                 '\.','').apply(len)>2]
    if not kk.empty:
        if DEBUG: print(unidecode.unidecode( l[i].get('NOMBRES').lower().split(' ')[0] ) )
        r=process.extractOne( unidecode.unidecode( l[i].get('NOMBRES').lower().split(' ')[0] )
                           ,  kk['WOS_author'].str.lower().apply(unidecode.unidecode).str.split(', ').str[-1],scorer=fuzz.partial_ratio )
        if r[1]==100 and r[2]==j:
            if DEBUG: print('found FN j=',r[2])
            l[i]=add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,ll,j)
            return l,i,ll,j

        if DEBUG: print(r)
        #if not 100 try second name: fail proof
        if DEBUG: print(unidecode.unidecode( l[i].get('NOMBRES').lower().split(' ')[-1]) ) 
        r=process.extractOne( unidecode.unidecode( l[i].get('NOMBRES').lower().split(' ')[-1] )
                           ,  kk['WOS_author'].str.lower().apply(unidecode.unidecode).str.split(', ').str[-1],scorer=fuzz.partial_ratio )
        if DEBUG: print(r)
        if r[1]==100 and r[2]==j:
            if DEBUG: print('found SN j=',r[2])
            l[i]=add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,ll,j)        
            return l,i,ll,j
    
    #if not 100 try first intial: fail proof
    if DEBUG: print(unidecode.unidecode( l[i].get('INICIALES').lower().replace('.','').split(' ')[0]) )
    r=process.extract( unidecode.unidecode( l[i].get('INICIALES').lower().replace('.','').split(' ')[0]) 
                       ,  ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                            ).str.replace('\.','').str.split(', ').str[-1],scorer=fuzz.partial_ratio )
    if DEBUG: print(r)
    rrr=[(rr[1],rr[2]) for rr in r if rr[2]==j]
    if rrr and rrr[0][0]==100:
        if DEBUG: print('found FI j=',rrr[0][1])
        l[i]=add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,ll,j)        
        return l,i,ll,j

    #if not 100 try second intial: fail proof
    if DEBUG: print(unidecode.unidecode( l[i].get('INICIALES').lower().replace('.','').split(' ')[-1]) )
    r=process.extract( unidecode.unidecode( l[i].get('INICIALES').lower().replace('.','').split(' ')[-1]) 
                       ,ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                        ).str.replace('\.','').str.split(', ').str[-1],scorer=fuzz.partial_ratio )
    if DEBUG: print(r)
    rrr=[(rr[1],rr[2]) for rr in r if rr[2]==j]
    if rrr and rrr[0][0]==100:
        if DEBUG: print('found SI j=',rrr[0][1])
        l[i]=add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,ll,j)
        return l,i,ll,j

    #Some times the first last are confused with the first names
    #'Luis Fernando Restrepo Betancur' → 'Luis Restrepo, B.''
    # One of the last name is already covered so only the initials are affected
    #if not 100 try second intial: fail proof
    r=process.extract( unidecode.unidecode( l[i].get('PRIMER APELLIDO').lower()[0] )
                       ,ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                        ).str.replace('\.','').str.split(', ').str[-1],scorer=fuzz.partial_ratio )
    if DEBUG: print(r)
    rrr=[(rr[1],rr[2]) for rr in r if rr[2]==j]
    if rrr and rrr[0][0]==100:
        if DEBUG: print('found SI j=',rrr[0][1])
        l[i]=add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,ll,j)
        return l,i,ll,j

    if l[i].get('SEGUNDO APELLIDO'):
        r=process.extract( unidecode.unidecode( l[i].get('SEGUNDO APELLIDO').lower()[0] ) 
                           ,ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                            ).str.replace('\.','').str.split(', ').str[-1],scorer=fuzz.partial_ratio )
        if DEBUG: print(r)
        rrr=[(rr[1],rr[2]) for rr in r if rr[2]==j]
        if rrr and rrr[0][0]==100:
            if DEBUG: print('found SI j=',rrr[0][1])
            l[i]=add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,ll,j)
            return l,i,ll,j
                      
    
    
    #print('final result: extract WOS_author and affiliauntion as lists:',lll[j])

    return l,i,ll,j

In [35]:
def missing_wos_authors_in_udea_authors(row):
    '''
    Compare UDEA_authors with authors_WOS and
    fill: 
    A) mising 'WOS_author' list in  UDEA_authors
    B) mising 'affiliation' list in  UDEA_authors
    1. Select first matching first last name
       a. If failed select first mathcing second last name
          with extra quality check in full name
    2. First mathching names
       a. If failed first matching initials
    '''    
    i=0
    j=-1
    l  =row['UDEA_authors']
    lll=row['authors_WOS']
    if lll:
        ll=pd.DataFrame(lll)
    else:
        ll=pd.DataFrame()
    #if not ll.empty:
    for i in range(len(l)):
        if not l[i].get('WOS_author'):
            l,i,ll,j=compare_names(l,i,ll,j,DEBUG=False)
    return l

In [36]:
UDEA['UDEA_authors']=UDEA.apply(missing_wos_authors_in_udea_authors,axis='columns')

Remove bad identified authors from the list in `UDEA_authors`

In [37]:
UDEA['Index']=list(UDEA.index)

In [38]:
def add_WOS_author_to_UDEA_authors(row_UDEA_authors_i,au,row_authors_WOS,iau):
    if not row_UDEA_authors_i.get('WOS_author'):
        row_UDEA_authors_i['WOS_author']=[au]
    else: #'WOS_author' list exists
        udeaiw=[unidecode.unidecode(s.lower().replace('.','').replace('-',' ')
                                   ) for s in  row_UDEA_authors_i['WOS_author']]
        if au not in udeaiw:
            #print('Check au not on WOS_author:',row_authors_WOS[iau].get('WOS_author'),row_UDEA_authors_i['WOS_author'])
            row_UDEA_authors_i['WOS_author'].append(row_authors_WOS[iau].get('WOS_author'))
    if not row_UDEA_authors_i.get('WOS_affiliation') and row_authors_WOS[iau].get('affiliation'):
        aff=row_authors_WOS[iau]['affiliation'] 
        row_UDEA_authors_i['WOS_affiliation']=aff
                
                
def Quality_UDEA_authors(row,maxlmin=74,DEBUG=False):
    '''
    Check the Quality for the match 
      UDEA_authors 
    with 
      authors_WOS
    '''
    #print('**',row['Index'])
    wos_list =[ unidecode.unidecode( d.get('WOS_author').lower(
                                     ).replace('.',''
                                     ).replace('-',' ')
                                   )
               for d in row.get('authors_WOS')]

    #If WOS_author not in row.get('UDEA_authors') is usually to be droppend
    
    
    udea_list=[ 
            [
              unidecode.unidecode( dd.lower(
                                     ).replace('.',''
                                     ).replace('-',' ')
                                 ) 
              for dd in d.get('WOS_author')
            ]
                if d.get('WOS_author') 
                else 
            [      
                unidecode.unidecode(
                  '{} {}, {}'.format(d.get('PRIMER APELLIDO'),
                                     d.get('SEGUNDO APELLIDO'),
                                     d.get('NOMBRES')
                                    ).replace(' ,',','
                                    ).strip(
                                    ).lower()
                  )
             ]
                  for d in row.get('UDEA_authors')
           ]    
    udea_list=[ list(set(l)) for l in udea_list  ]
    
    if not udea_list:
        if DEBUG: print('==1==')
        Q=True
        return Q
    for i in range(len(udea_list)): #=len(row['UDEA_authors'])
    #for l in udea_list: #=len(row['UDEA_authors'])
        Q=False
        for au in wos_list:
            #Exact match
            r=np.intersect1d( [au],udea_list[i] ).shape[0]
            if r>0:
                if DEBUG: print('==2==')
                Q=True
                break
            else:
                #Similarity match
                #print(au,udea_list[i])
                rr=process.extractOne(au,udea_list[i],scorer=fuzz.partial_ratio)
                if rr[1]==100:
                    kk=add_WOS_author_to_UDEA_authors(row['UDEA_authors'][i],au,
                                                      row['authors_WOS'],wos_list.index(au) )                                        
                    if DEBUG: print('==3==')
                    Q=True
                    break
                #print(rr)
                rr=process.extractOne(au,udea_list[i],scorer=fuzz.token_set_ratio)
                if rr[1]==100:
                    kk=add_WOS_author_to_UDEA_authors(row['UDEA_authors'][i],au,
                                                      row['authors_WOS'],wos_list.index(au) )                                        
                    if DEBUG: print('==4==')
                    Q=True
                    break
                #keep only the first letter of names
                fnau=re.sub('([\w\s]+\,\s\w)[\w\s]+',r'\1',au)
                l=[re.sub('([\w\s]+\,\s\w)[\w\s]+',r'\1',ll) for ll in udea_list[i]]
                #print(l)
                rr=process.extractOne(fnau,l,scorer=fuzz.token_set_ratio)
                if rr[1]==100:
                    kk=add_WOS_author_to_UDEA_authors(row['UDEA_authors'][i],au,
                                                      row['authors_WOS'],wos_list.index(au) )                    
                    if DEBUG: print('==5==')
                    Q=True
                    break
                    
                #keep only first letter of second last name
                l=[re.sub('(\s\w)\w+\,',r'\1,',ll) for ll in udea_list[i]]
                #print(l)
                rr=process.extractOne(au,l,scorer=fuzz.token_set_ratio)
                if rr[1]==100:
                    kk=add_WOS_author_to_UDEA_authors(row['UDEA_authors'][i],au,
                                                      row['authors_WOS'],wos_list.index(au) )                    
                    if DEBUG: print('==6==')
                    Q=True
                    break

                #remove first last name from udea_list[i] and the first letter of names
                l=[re.sub('^\w+\s(\w\w+\,\s\w)\w+',r'\1',ll) for ll in udea_list[i]]
                #print(l)
                rr=process.extractOne(au,l,scorer=fuzz.token_set_ratio)
                if rr[1]==100:
                    kk=add_WOS_author_to_UDEA_authors(row['UDEA_authors'][i],au,
                                                      row['authors_WOS'],wos_list.index(au) )                    
                    if DEBUG: print('==7==')
                    Q=True
                    break
                #Very similar names in any order
                #print('*',au,udea_list[i])
                rr=process.extractOne(au,udea_list[i],scorer=fuzz.token_sort_ratio)
                if rr[1]>95:
                    kk=add_WOS_author_to_UDEA_authors(row['UDEA_authors'][i],au,
                                                      row['authors_WOS'],wos_list.index(au) )                    
                    if DEBUG: print('==8==')
                    Q=True
                    break
                #Keep only first last name and the first letter of names
                fnau= re.sub('(^\w\w+)\s\w+(\,\s\w)[\s\w]*',r'\1\2',au)
                l   =[re.sub('(^\w\w+)\s\w+(\,\s\w)[\s\w]*',r'\1\2',ll) for ll in udea_list[i]]
                #print(l)
                rr=process.extractOne(fnau,l,scorer=fuzz.token_set_ratio)
                if rr[1]==100:
                    kk=add_WOS_author_to_UDEA_authors(row['UDEA_authors'][i],au,
                                                      row['authors_WOS'],wos_list.index(au) )                    
                    if DEBUG: print('==9==')
                    Q=True
                    break
                
                #Q=False
        if not Q:
            maxl=0
            for j in range(len(udea_list[i])):
                rr=process.extractOne(udea_list[i][j],wos_list,scorer=fuzz.ratio)
                if DEBUG: print(rr)
                if rr[1]>maxl:
                    maxl=rr[1]
            if DEBUG: print(udea_list[i],i,j,maxl)
            #keep if maxl>maxlmin See step: ==10===
            #print('*',row['UDEA_authors'],i)
            if maxl>maxlmin:
                #udea_list[i][j],wos_list
                #print('Check rr[0] not on WOS_author:',rr[0],row['UDEA_authors'][i].get('WOS_author'))

                kk=add_WOS_author_to_UDEA_authors(row['UDEA_authors'][i],rr[0],
                                                      row['authors_WOS'],wos_list.index(rr[0]) )
                if DEBUG: print('==10==')
                Q=True
                return Q
            else:  #Remove and exit
                row['UDEA_authors'].pop(i)
                return Q
            
    return Q

In [39]:
#i=7
#UDEA.loc[[i]].apply(lambda row:Quality_UDEA_authors(row,DEBUG=True),axis='columns')

In [40]:
kold=pd.np.inf
while True:
    Q=UDEA.apply(Quality_UDEA_authors,axis='columns')
    kk=UDEA[~Q].shape[0]
    print(kk)
    if kk==0 or kk==kold:
        kold=kk
        UDEA['NEW_Q']=Q
        break

1
0


Generate new `NEW_authors` column with full authors in JSON format. ONLY AFTER PREVIOUS NORMALIZATIONS

In [41]:
def add_wos(d):
    dd={}
    for k, v in d.items():
        if k=='affiliation':
            dd['WOS_{}'.format(k)]=v
        elif k=='WOS_author':
            dd[k]=[v]
        else:
            dd[k]=v
    return dd
    
def authors(row,DEBUG=False):
    '''
    Check the Quality for the match 
      UDEA_authors 
    with 
      authors_WOS
    '''
    ll=[]
    if not row['NEW_Q']:
        print('WARNING: bad quality record!')
        return ll

    wos_list =[  d.get('WOS_author') for d in row.get('authors_WOS')]
    
    udea_list=[  [dd for dd in d.get('WOS_author')]
                   if d.get('WOS_author') 
                   else []
                for d in row.get('UDEA_authors') ]
    
    udea_list=[ list(set(l)) for l in udea_list  ]
    
    if not udea_list:
        for d in row['authors_WOS']:
            ll.append(add_wos(d))
        if DEBUG: print('==1==')
        return ll

    for i in range(len(wos_list)):
        dd={}
        ADD=True
        for j in range(len(udea_list)): #=len(row['UDEA_authors'])
            r=np.intersect1d( [wos_list[i]],udea_list[j] ).shape[0]
            if r>0:
                ADD=False
                ll.append( row['UDEA_authors'][j].copy()  )
                if DEBUG: print('==2==')                
                break
        if ADD:
            ll.append(add_wos( row['authors_WOS'][i] ) )
            if DEBUG: print('==2==')
            
    return ll

In [42]:
#i=1
#UDEA.loc[[i]].apply(lambda row:authors(row,DEBUG=True),axis='columns').loc[i]
#UDEA.loc[[i]].apply(lambda row:authors(row,DEBUG=True),axis='columns').loc[i]
#UDEA.loc[i,'authors_WOS']#[0]['WOS_author']='manuel, j'

In [43]:
UDEA['NEW_authors']=UDEA.apply(authors,axis='columns')

In [44]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(lambda l: l if l else [])

Checks!

In [45]:
UDEA[UDEA['UDEA_authors'].apply(lambda l: len(l)>0)].shape

(12110, 184)

In [46]:
UDEA[UDEA['NEW_authors'].apply(lambda l: len(l)>0)].shape

(15642, 184)

Other normalizations

In [47]:
UDEA['DI']=UDEA.apply( 
     lambda row: row['DI'] if row['DI'] else row['SCP_DOI'],axis=1 ).str.strip().str.lower()
UDEA['DI']=UDEA.apply( 
     lambda row: row['DI'] if row['DI'] else row['SCI_DI'],axis=1 ).str.strip().str.lower()

In [48]:
def normalize_NAME(row,WOS_NAME,SCI_NAME,SCP_NAME,UDEA_NAME,
                   SCI_sub=('',''),SCP_sub=('',''),UDEA_sub=('',''),
                  DEBUG=False):
    '''
    If the WOS of Science column is empty, copy the
    data from a filled column either from 
    * SCI
    * SCP
    * UDEA
    '''
    sn=row[WOS_NAME]
    if not sn:
        if SCI_NAME and row[SCI_NAME]:
            sn=re.sub( SCI_sub[0],SCI_sub[1], str( row[SCI_NAME] ) )
            if DEBUG:
                print('SCI: {}:{}'.format(row[SCI_NAME], sn))
        elif SCP_NAME and row[SCP_NAME]:
            sn=re.sub( SCP_sub[0],SCP_sub[1], str( row[SCP_NAME] ) )
        elif UDEA_NAME and row[UDEA_NAME]:
            sn=re.sub( SCP_sub[0],SCP_sub[1], str( row[UDEA_NAME] ) )
            if DEBUG:
                print('UDEA: {}:{}'.format(row[UDEA_NAME], sn))            
    return sn

In [49]:
WOS_NAME ='SN'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_SN',
                                      SCP_NAME ='SCP_ISSN',
                                      UDEA_NAME='UDEA_issn rev',
                                      SCP_sub=( '^([\w]{4})([\w]{4})$',r'\1-\2\n'  ),
                                      UDEA_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [50]:
WOS_NAME ='DI'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_DI',
                                      SCP_NAME ='SCP_DOI',
                                      UDEA_NAME='UDEA_doi')
                      ,axis='columns')

In [51]:
WOS_NAME ='VL'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_VL',
                                      SCP_NAME ='SCP_Volume',
                                      UDEA_NAME='',
                                      SCI_sub=('(\w)$',r'\1\n'),
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [52]:
WOS_NAME ='IS'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_IS',
                                      SCP_NAME ='SCP_Issue',
                                      UDEA_NAME='',
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [53]:
WOS_NAME ='BP'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_BP',
                                      SCP_NAME ='SCP_Page start',
                                      UDEA_NAME='',
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [54]:
WOS_NAME ='EP'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_EP',
                                      SCP_NAME ='SCP_Page end',
                                      UDEA_NAME='',
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [55]:
UDEA['UDEA_idioma']=UDEA['UDEA_idioma'].apply(lambda s: 'Spanish' if s=='ESPA?' else s)
UDEA['UDEA_idioma']=UDEA['UDEA_idioma'].apply(lambda s: 'English' if s=='INGLE' else s)
WOS_NAME ='LA'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_LA',
                                      SCP_NAME ='SCP_Language of Original Document',
                                      UDEA_NAME='UDEA_idioma',
                                      SCI_sub =('\t',r''),
                                      SCP_sub =('(\w)$',r'\1\n'),
                                      UDEA_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

# Export to other formats

In [56]:
udea=UDEA.copy()
udea=udea[udea['UDEA_authors'].apply(lambda l: len(l)>0)].reset_index(drop=True)

In [57]:
def convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(df,
                            json_column='UDEA_authors',dictionary_key='FACULTAD',sep='; '):
    return df[json_column].apply(lambda l:  
       [d.get(dictionary_key) for d in  l if type(l)==list and d.get(dictionary_key) ]  
               ).apply(
        pd.np.unique
        ).apply(sep.join)    

In [58]:
JSON=True
if not JSON:
    udea['FACULTAD']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                            json_column='UDEA_authors',dictionary_key='FACULTAD')
    udea['DEPARTAMENTO']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                            json_column='UDEA_authors',dictionary_key='DEPARTAMENTO')
    udea['GRUPO']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                            json_column='UDEA_authors',dictionary_key='GRUPO')
    udea['autores_UDEA']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                            json_column='UDEA_authors',dictionary_key='full_name')
if JSON:
    año='annio'; título='tituloArticulo'; revista='nombreRevista'
    issn='issn'; doi='doi'; volumen='volumen'; número='numero'
    páginas='paginas'; semestre='semestre'; idioma='idioma'
    tipoRevista='tipoRevista';     Colciencias='clasificacionColciencias'
    proyecto='proyectoAsociado'
    autores='autores'
else:
    año='Año';   título='Título del Artículo'; revista='Nombre de la Revista'
    issn='ISSN'; doi='DOI'; volumen='Volumen'; número='numero'
    páginas='Páginas'; semestre='Semestre'; idioma='Idioma'
    tipoRevista='Tipo de Revista'; Colciencias='Clasificación Colciencias'
    proyecto='Producto asociado a un proyecto de extensión o investigación'
    udea['Nombre del proyecto de extensión o investigación']=''
    autores='Autores'    

## Prepare columns

In [59]:
f=open('example.json','w')
f.write(
'''
[{
  "autores" : [ 
      {
          "nombreAutor" : "Jonathan Aguilar Bedoya",
          "identificacionAutor" : null,
          "tipoAutor" : "Estudiante de posgrado",
          "programaAcademico" : [ 
              "60178-MAESTRÍA EN INGENIERÍA MECÁNICA-INVESTIGACIÓN"
          ],
          "grupoInvestigacion" : [ 
              "Grupo de Energía Alternativa"
          ]
      }, 
      {
          "nombreAutor" : "Ainhoa Rubio Celemente",
          "tipoAutor" : "Autor externo",
          "grupoInvestigacion" : [ 
              "Grupo de Energía Alternativa"
          ],
          "institucion" : "Tecnológico de Antioquia. Institución Universitaria",
          "pais" : "Colombia"
      }, 
      {
          "nombreAutor" : "Laura Isable Velasquez Garcia",
          "tipoAutor" : "Profesor ocasional",
          "programaAcademico" : [ 
              "516-INGENIERÍA MECÁNICA"
          ],
          "grupoInvestigacion" : [ 
              "Grupo de Energía Alternativa"
          ]
      }, 
      {
          "nombreAutor" : "Edwin Lenin Chica Arrieta",
          "identificacionAutor" : "...",
          "tipoAutor" : "Profesor vinculado",
          "programaAcademico" : [ 
              "516-INGENIERÍA MECÁNICA"
          ],
          "grupoInvestigacion" : [ 
              "Grupo de Energía Alternativa"
          ]
      }
  ],
  "tituloArticulo" : "Design and Optimization of a Multi-Element Hydrofoil for a Horizontal-Axis Hydrokinetic Turbine",
  "nombreRevista" : "Energies",
  "issn" : "1996-1073",
  "doi" : "10.3390/en12244679",
  "volumen" : 12,
  "numero" : 24,
  "paginas" : "1-18",
  "annio" : 2019,
  "semestre" : "2019-2",
  "idioma" : "Inglés",
  "tipoRevista" : "Revista Internacional Indexada",
  "clasificacionColciencias" : "A1",
  "proyectoAsociado" : null
}
]
'''
)
f.close()

In [60]:
def new_Autores(l,nombre='nombreAutor',tipo='tipoAutor',identificación='identificacionAutor',
                institución='institucion',país='pais',grupo='grupoInvestigacion',
                facultad='facultad',departamento='departamento'):
    ll=[]
    for d in l:
        dd={}
        if d.get('CÉDULA'):
            dd[identificación]=int(d['CÉDULA'])

        if d.get('NOMBRE COMPLETO') and d.get('full_name'):
            dd[nombre]=d['NOMBRE COMPLETO']
            dd[tipo]='Profesor vinculado'
            dd[institución]='Universidad de Antioquia'
            dd[país]='Colombia'
        else:
            if d.get('WOS_author'):
                l=d['WOS_author'][0].split(', ')
                dd[nombre]=l[-1]+' '+l[0]
            if d.get('WOS_affiliation'):
                dd[institución]=d['WOS_affiliation'][0]
        if d.get('GRUPO'):
            dd[grupo]=[d['GRUPO']]
        if d.get('DEPARTAMENTO'):
            dd[departamento]=d['DEPARTAMENTO']
        if d.get('FACULTAD'):
            dd[facultad]=d['FACULTAD']
            
        ll.append(dd)
    return ll

In [61]:
udea[autores]=udea['NEW_authors'].apply(new_Autores)

In [62]:
udea[título]=udea['TI']
udea[revista]=udea['SO'].replace('\n',' ')
udea[issn]=udea['SN'].str.replace('\n','')
udea[doi]=udea['DI'].apply(lambda s: s if s else None)
udea[volumen]=udea['VL'].str.replace('\n','').apply(lambda s: s if s else None)
udea[número]=udea['IS'].str.replace('\n','')
udea[páginas]=udea.apply(lambda row:
                  row['BP'].replace('\n','')+'-'+row['EP'].replace('\n','') 
                  if row['BP'] 
                  else row['BP'],axis='columns').str.replace('\-$','')
udea[año]=udea['PY']
udea[semestre]=None
udea[idioma]=udea['LA'].str.replace('\n','')
udea[tipoRevista]=udea.apply(lambda row: 'Revista Internacional Indexada' 
           if row.get('Tipo').find('WOS')>-1 or row.get('Tipo').find('SCP')
           else 'Revista Indexada en SCIELO',axis='columns')
udea[Colciencias]=udea['UDEA_tipo mat'].str.replace('^\w+([ABC][12]*)$',r'\1' ).apply(
                       lambda s:s if len(s)<3 else None)
udea[proyecto]=None

In [63]:
if JSON:
    new_format=pd.read_json( 'example.json' )
else:
    new_format=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vTiaxuZSGmI-aFgMUVjRAU3ws7WN9xmtjMEWu_SLOd5kAq_ZAuUtJUVr8qxNl3sMcp_fjE2gLVt_tdp/pub?output=xlsx')

In [64]:
cls=["autores","tituloArticulo","nombreRevista","issn","doi","volumen","numero",
     "paginas","annio","semestre","idioma","tipoRevista",
     "clasificacionColciencias","proyectoAsociado"]

In [65]:
pd.np.setdiff1d(cls,list(new_format.columns))

array([], dtype='<U24')

In [66]:
udea[cls][:1]

,autores,tituloArticulo,nombreRevista,issn,doi,volumen,numero,paginas,annio,semestre,idioma,tipoRevista,clasificacionColciencias,proyectoAsociado
0,"[{'institucion': 'Universidad de Antioquia', 'nombreAutor': 'Edwin Garcia Quintero', 'tipoAutor': 'Profesor vinculado', 'grupoInvestigacion': ['Grupo de Investigación en Materiales y Sistemas Ener...",Methodology for evaluation of voltage sags,REVISTA FACULTAD DE INGENIERIA-UNIVERSIDAD DE ANTIOQUIA,0120-6230,None,None,53,196-204,2010,None,Spanish,Revista Internacional Indexada,A1,None


In [67]:
udea[cls].to_json('udea.json',orient='records',force_ascii=False)

tests

In [ ]:
kk=pd.read_json('udea.json')

In [2]:
import json
with open(r"udea.json", "r") as read_file:
    data = json.load(read_file)

# NOTA
También se ha añadido 
* `departamento`: Dependencia de la Facultad a la que está adscrito el profesor

In [111]:
import pandas as pd

In [119]:
import getpass

In [122]:
idrive=getpass.getpass('Google Drive id:\n')

Google Drive id:
 ········································································


In [124]:
df=pd.read_excel('https://docs.google.com/spreadsheets/d/e/{}/pub?output=xlsx'.format(idrive),
                skiprows=3)

In [224]:
foreign=df[df['CEDULA']<999999]

In [231]:
foreign[foreign['NOMBRE'].str.split().apply(len)==3]['NOMBRE'].reset_index(drop=True)

0                FISCHLER PAUL HENRY
1                   RUSEV ILKO SEMOV
2           BAUTISTA VIGUERAS XANATH
3                 NAGAR DAYA KRISHNA
4                ROMANO ANTONIO ENEA
5                    BOCK BRIAN CARL
6             MUÑOZ QUESADA FERNANDO
7                SMITH JAMES ANTHONY
8              ROSIQUE GRACIA JAVIER
9           JIMENEZ BENITEZ MERCEDES
10          QUEREJAZU ESCOBARI AMAYA
11                       JANG JI SON
12            FRIEDERICI LUISA MARIE
13          MUSIALEK STYCZEN DANIELE
14            MCNULTY MARIA MARGARET
15                    RABE ANA MARIA
16                 RAMOS POLLAN RAUL
17              ACIN MARTINEZ SERGIO
18    KAMBOUROVA MIGLENA GUEORGUIEVA
19               VILLAR ARGAIZ DAVID
20             RODRIGUEZ RUIZ YORDAN
Name: NOMBRE, dtype: object

In [129]:
df['NOMBRE'].apply( lambda s: [ ss for ss in s.split() if len(ss)<=3]).apply(pd.Series).stack().unique()

array(['DEL', 'DE', 'LA', 'PAZ', 'ANA', 'LUZ', 'REY', 'GIL', 'SUS', 'ANI',
       'ELI', 'VON', 'EVA', 'JI', 'SON', 'NEL', 'ZEA', 'LOS', 'ROA',
       'MAR', 'L', 'A', 'MC', 'RUA', 'G', 'LIA', 'JES', 'LEW', 'SAN',
       'RIO', 'LIZ', 'S', 'EDU'], dtype=object)

In [180]:
sorted( ['PAZ','REY', 'GIL', 'SUS',
         'ZEA',  'ROA',
        'RUA',   'LEW',
       'RIO', 'LIZ'])

['GIL', 'LEW', 'LIZ', 'PAZ', 'REY', 'RIO', 'ROA', 'RUA', 'SUS', 'ZEA']

In [177]:
df[df['NOMBRE'].str.contains('PAZ')]['NOMBRE']

27    CAMISON PAZ JOSE LUIS
Name: NOMBRE, dtype: object

In [202]:
    import re
    def split_names(s,exceptions=['GIL','LEW','LIZ','PAZ','REY','RIO','ROA','RUA','SUS','ZEA']):
        """
        Extract the parts of the full name `s` in the format ([] → optional):

        [SMALL_CONECTORS] FIRST_LAST_NAME [SMALL_CONECTORS] [SECOND_LAST_NAME] NAMES

        * If len(s) == 2 → Foreign name assumed with single last name on it
        * If len(s) == 3 → Colombian name assumed two last mames and one first name

        Add short last names to `exceptions` list if necessary

        Works with:
        ----
            s='LA ROTTA FORERO DANIEL ANDRES'
            s='MONTES RAMIREZ MARIA DEL CONSUELO'
            s='CALLEJAS POSADA RICARDO DE LA MERCED'
            s='DE LA CUESTA BENJUMEA MARIA DEL CARMEN'
            s='JARAMILLO OCAMPO NICOLAS CARLOS MARTI'
            s='RESTREPO QUINTERO DIEGO ALEJANDRO'
            s='RESTREPO ZEA JAIRO HUMBERTO'
            s='JIMENEZ DEL RIO MARLEN'        
            s='RESTREPO FERNÁNDEZ SARA' # Colombian: two LAST_NAMES NAME
            s='NARDI ENRICO' # Foreing
        Fails:
        ----
            s='RANGEL MARTINEZ VILLAL ANDRES MAURICIO' # more than 2 last names
            s='ROMANO ANTONIO ENEA' # Foreing → LAST_NAME NAMES
        """
        s=s.title()
        exceptions=[e.title() for e in exceptions]
        sl=re.sub('(\s\w{1,3})\s',r'\1-',s,re.UNICODE)
        sl=re.sub('(\s\w{1,3}\-\w{1,3})\s',r'\1-',sl,re.UNICODE)
        sl=re.sub('^(\w{1,3})\s',r'\1-' ,sl,re.UNICODE)
        #Clean exceptions
        #Extract short names list
        lst=[s for s in re.split( '(\w{1,3})\-',sl ) if len(s)>=1 and len(s)<=3 ]
        #intersection with exceptions list
        exc=[value for value in exceptions if value in lst]
        if exc:
            for e in exc:
                sl=sl.replace('{}-'.format(e),'{} '.format(e))

        #if sl.find('-')>-1:
        print(sl)
        sll=[s.replace('-',' ') for s in sl.split()]
        if len(s.split())==2:
            sll=[s.split()[0]]+['']+[s.split()[1]]
        #
        d={'NOMBRE COMPLETO' : ' '.join(sll[2:]+sll[:2]),
         'PRIMER APELLIDO' : sll[0], 
         'SEGUNDO APELLIDO': sll[1], 
         'NOMBRES'         :' '.join(sll[2:]), 
         'INICIALES'       :' '.join( [i[0]+'.' for i in ' '.join(sll[2:]).split() ] )
        }
        return d

In [203]:
split_names('JIMENEZ DEL RIO MARLEN')

Jimenez Del-Rio Marlen


{'INICIALES': 'M.',
 'NOMBRE COMPLETO': 'Marlen Jimenez Del Rio',
 'NOMBRES': 'Marlen',
 'PRIMER APELLIDO': 'Jimenez',
 'SEGUNDO APELLIDO': 'Del Rio'}

In [204]:
split_names('RESTREPO ZEA JAIRO HUMBERTO')

Restrepo Zea Jairo Humberto


{'INICIALES': 'J. H.',
 'NOMBRE COMPLETO': 'Jairo Humberto Restrepo Zea',
 'NOMBRES': 'Jairo Humberto',
 'PRIMER APELLIDO': 'Restrepo',
 'SEGUNDO APELLIDO': 'Zea'}

In [205]:
split_names('YARZA DE LOS RIOS VICTOR ALEXANDER')

Yarza De-Los-Rios Victor Alexander


{'INICIALES': 'V. A.',
 'NOMBRE COMPLETO': 'Victor Alexander Yarza De Los Rios',
 'NOMBRES': 'Victor Alexander',
 'PRIMER APELLIDO': 'Yarza',
 'SEGUNDO APELLIDO': 'De Los Rios'}

In [207]:
ns='''GOMEZ ISAZA MARIA CRISTINA DEL ROSARIO
KAJSIU BLENDI
LOPEZ GOMEZ MARIA DEL SOCORRO
DIAZ WALTER
YARZA DE LOS RIOS VICTOR ALEXANDER
BERNAL DE JARAMILLO LUCIA VICTORIA
RAMIREZ ARISTEGUIETA LUIS MIGUEL RAMON
ALVAREZ MARTINEZ EFRAIN DEL CRISTO
GALEANO JARAMILLO ELKIN DE JESUS
CONTRERAS CALDERON JOSE DEL CARMEN
GAMEZ DE ARMAS GUSTAVO ADOLFO
RODRIGUEZ GAZQUEZ MARIA DE LOS ANGELES
GRISALES ROMERO HUGO DE JESUS
ALZATE YEPES TERESA DE JESUS
RESTREPO FERNÁNDEZ SIMÓN
ROMANO ANTONIO ENEA
JIMENEZ DEL RIO MARLEN
RESTREPO ZEA JAIRO HUMBERTO'''.split('\n')

In [208]:
dn=[]
for n in ns:
    dn.append( split_names(n))

Gomez Isaza Maria Cristina Del-Rosario
Kajsiu Blendi
Lopez Gomez Maria Del-Socorro
Diaz Walter
Yarza De-Los-Rios Victor Alexander
Bernal De-Jaramillo Lucia Victoria
Ramirez Aristeguieta Luis Miguel Ramon
Alvarez Martinez Efrain Del-Cristo
Galeano Jaramillo Elkin De-Jesus
Contreras Calderon Jose Del-Carmen
Gamez De-Armas Gustavo Adolfo
Rodriguez Gazquez Maria De-Los-Angeles
Grisales Romero Hugo De-Jesus
Alzate Yepes Teresa De-Jesus
Restrepo Fernández Simón
Romano Antonio Enea
Jimenez Del-Rio Marlen
Restrepo Zea Jairo Humberto


In [209]:
dn

[{'INICIALES': 'M. C. D. R.',
  'NOMBRE COMPLETO': 'Maria Cristina Del Rosario Gomez Isaza',
  'NOMBRES': 'Maria Cristina Del Rosario',
  'PRIMER APELLIDO': 'Gomez',
  'SEGUNDO APELLIDO': 'Isaza'},
 {'INICIALES': 'B.',
  'NOMBRE COMPLETO': 'Blendi Kajsiu ',
  'NOMBRES': 'Blendi',
  'PRIMER APELLIDO': 'Kajsiu',
  'SEGUNDO APELLIDO': ''},
 {'INICIALES': 'M. D. S.',
  'NOMBRE COMPLETO': 'Maria Del Socorro Lopez Gomez',
  'NOMBRES': 'Maria Del Socorro',
  'PRIMER APELLIDO': 'Lopez',
  'SEGUNDO APELLIDO': 'Gomez'},
 {'INICIALES': 'W.',
  'NOMBRE COMPLETO': 'Walter Diaz ',
  'NOMBRES': 'Walter',
  'PRIMER APELLIDO': 'Diaz',
  'SEGUNDO APELLIDO': ''},
 {'INICIALES': 'V. A.',
  'NOMBRE COMPLETO': 'Victor Alexander Yarza De Los Rios',
  'NOMBRES': 'Victor Alexander',
  'PRIMER APELLIDO': 'Yarza',
  'SEGUNDO APELLIDO': 'De Los Rios'},
 {'INICIALES': 'L. V.',
  'NOMBRE COMPLETO': 'Lucia Victoria Bernal De Jaramillo',
  'NOMBRES': 'Lucia Victoria',
  'PRIMER APELLIDO': 'Bernal',
  'SEGUNDO APELLID

## Perfiles de Google Scholar

In [1]:
import pandas as pd

In [2]:
#https://drive.google.com/file/d/1PBXwl2uHx8beDEduq7KdDNqN3WD14TKg/view?usp=sharing
df=pd.read_json('/home/restrepo/Downloads/udea.json')

In [3]:
df[df.autores.apply( lambda l: 'Facultad de Ciencias Exactas y Naturales' in [d.get('facultad') for d in l] if isinstance(l,list) else False   )].shape

(2243, 14)

In [4]:
#https://drive.google.com/file/d/1aSf6eRpnH1bPG0AyNCsrQsEuHNntn0oY/view?usp=sharing
gs=pd.read_json('/home/restrepo/Downloads/gsudea.json.gz')

In [5]:
kk=gs.dropna(subset=['DOI'])
kk=kk[kk['DOI']!=''].reset_index(drop=True)
kk['doi']=kk['DOI'].str.lower()
df['doi']=df['doi'].str.lower()

In [6]:
kk=kk[['doi','profiles']].merge(df[['doi','autores']],on='doi',how='left').reset_index(drop=True)

In [7]:
kk.shape

(7990, 3)

In [8]:
facultad='Facultad de Ciencias Exactas y Naturales' 
fc=kk[kk['autores'].apply( lambda l: facultad in [d.get('facultad') for d in l] if isinstance(l,list) else False   )].reset_index(drop=True)

In [9]:
fc['fc_autores']=fc['autores'].apply(lambda l: [{d.get('departamento'):d.get('nombreAutor')}  for d in l if d.get('facultad')==facultad])

In [36]:
gs_ids=[]
profiles=[]
authors=[]
dauthors=[]
for i in fc.index:
    row=fc.loc[i]
    for d in row['fc_autores']:
        if d not in authors:
            authors.append(d)
            dauthors.append({'affiliation':d})
        if isinstance( row['profiles'],dict):
            for k in row['profiles'].keys():
                if k.split()[-1] in list(d.values())[0].split():
                    if row['profiles'].get(k) not in gs_ids:
                        gs_ids.append(row['profiles'].get(k))
                        profiles.append({'author':k,'affiliation':d,'gs_id':row['profiles'].get(k)})

In [39]:
pf=pd.DataFrame( profiles )
pf['full_name']=pf['affiliation'].apply(lambda d: list(d.values())[0] )
pf['departamento']=pf['affiliation'].apply(lambda d: list(d.keys())[0] )
pf=pf.drop('affiliation',axis='columns').sort_values('departamento').reset_index(drop=True)
pf=pf.drop_duplicates('full_name').reset_index(drop=True)
pf

,author,gs_id,full_name,departamento
0,LA Valencia,wQvZ7ZcAAAAJ,Leon Alexander Valencia Henao,Departamento de Matemáticas
1,H Montegranario,pOFC9ogAAAAJ,Hebert Montegranario Riascos,Departamento de Matemáticas
2,G Bedoya,FhJ_shsAAAAJ,Gabriel De Jesus Bedoya Berrio,Instituto de Biología
3,O Triana,Y1odF2kAAAAJ,Omar Triana Chavez,Instituto de Biología
4,S Solari,0CMLqh4AAAAJ,Sergio Alcides Solari Torres,Instituto de Biología
5,JF Blanco,Le6VqVwAAAAJ,Juan Felipe Blanco Libreros,Instituto de Biología
6,JM Daza,Fpc3D0UAAAAJ,Juan Manuel Daza Rojas,Instituto de Biología
7,M Wolff,onbpsLYAAAAJ,Marta Isabel Wolff Echeverri,Instituto de Biología
8,JL Parra,USDYbtIAAAAJ,Juan Luis Parra Vergara,Instituto de Biología
9,ID Soto,RxIOe6oAAAAJ,Ivan Dario Soto Calderon,Instituto de Biología


In [45]:
pf[pf['departamento']=='Instituto de Física'].reset_index(drop=True)

,author,gs_id,full_name,departamento
0,R Henao,-aMybj4AAAAJ,Rodrigo De Jesus Henao Henao,Instituto de Física
1,JE Mahecha,8rFNfD8AAAAJ,Jorge Eduardo Mahecha Gomez,Instituto de Física
2,O Arnache,R5OkApAAAAAJ,Oscar Luis Arnache Olmos,Instituto de Física
3,J Jaramillo,6cTMyi8AAAAJ,Johny Alexander Jaramillo Gallego,Instituto de Física
4,LA Pachon,Z1x3w3MAAAAJ,Leonardo Augusto Pachon Contreras,Instituto de Física
5,PA Cuartas,c4zrU20AAAAJ,Pablo Andres Cuartas Restrepo,Instituto de Física
6,JI Zuluaga,qpGVqNwAAAAJ,Jorge Ivan Zuluaga Callejas,Instituto de Física
7,JA Osorio,_TYCf_wAAAAJ,Jaime Alberto Osorio Velez,Instituto de Física
8,MA Giraldo,SpLyrvoAAAAJ,Marco Antonio Giraldo Cadavid,Instituto de Física
9,E Rueda,VFpEf0YAAAAJ,Edgar Alberto Rueda Munoz,Instituto de Física


In [42]:
pfa=pd.DataFrame( dauthors )
pfa['full_name']=pfa['affiliation'].apply(lambda d: list(d.values())[0] )
pfa['departamento']=pfa['affiliation'].apply(lambda d: list(d.keys())[0] )
pfa=pfa.drop('affiliation',axis='columns').sort_values('departamento').reset_index(drop=True)
pfa

,full_name,departamento
0,Carlos Alberto Marin Arango,Departamento de Matemáticas
1,Luis Fernando Echeverri Delgado,Departamento de Matemáticas
2,Carlos Mario Jaramillo Lopez,Departamento de Matemáticas
3,Raul Eduardo Velasquez Ossa,Departamento de Matemáticas
4,Johanna Marcela Orozco Castañeda,Departamento de Matemáticas
...,...,...
133,Maria Victoria Alzate Cano,Instituto de Química
134,Betty Lucy Lopez Osorio,Instituto de Química
135,Leon Felipe Otalvaro Tamayo,Instituto de Química
136,Wilson Isidro Cardona Galeano,Instituto de Química


In [46]:
pfa[pfa['departamento']=='Instituto de Física'].reset_index(drop=True)

,full_name,departamento
0,Boris Anghelo Rodriguez Rey,Instituto de Física
1,Jorge Eduardo Mahecha Gomez,Instituto de Física
2,Jorge Enrique Tobon Gomez,Instituto de Física
3,Oscar Luis Arnache Olmos,Instituto de Física
4,Oscar Alberto Zapata Noreña,Instituto de Física
5,Rodrigo De Jesus Henao Henao,Instituto de Física
6,Cesar Augusto Barrero Meneses,Instituto de Física
7,Jorge Mario Osorio Guillen,Instituto de Física
8,Ignacio Ramon Ferrin Vazquez,Instituto de Física
9,John Fredy Barrera Ramirez,Instituto de Física
